In [16]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd


Le Covid-19 peut être modélisé à l'aide d'un modèle de type Susceptible-Exposed-Infectious-Removed (SEIR).
Le package EoN (Epidemics on Networks) permet justement de créer ce genre d'estimation.

On estime qu'en France une personne adulte a en moyenne un contact avec 40 autres personnes et que le virus est à un taux de reproduction (R_0) estimé entre 1.4 et 3.2 (https://cmmid.github.io/topics/covid19/current-patterns-transmission/global-time-varying-transmission.html). 

Sa période d'incubation est de 5 jours et la période de contagion est d'environ 11 jours.


Le modèle SEIR est un modèle permettant de voir comment une maladie se répend dans la population. 

L'acronyme tient pour : Suceptible Exposed Infected Recovered (Suspicision Exposé Infecté Rétabli)


<center>
$\dot{S}  = \beta SI$ (1)
    
    Variation de personnes suceptible de tomber malade, modéré par le nombre de personnes infectés et leur contact avec les infectés
</center>
<center>
$\dot{E} = \beta SI - \alpha E $  (2)
    
    Nombre de personnes ayant été exposé à la maladie. Croissance basé sur le taux de contact mais diminuer par la période d'incubation 
</center>
<center>
$\dot{I} = \alpha E - \gamma I$ (3)
    
    Variation dans le nombre de personnes infectées basé sur la population exposée and la période d'incubation. Décroit en fonction de la période d'infection, donc plus $\gamma $ est élévée plus les gens meurent vite ou se restaurent vite ce qui nous envoie vers R le nombre de personne ayant été soigné/restauré.
</center>
<center>
$\dot{R} \gamma I$ (4)
    
    
</center>
<center>
$N = S + E + I +R$ (5)
    
    Contrainte pour l'assurer que la population reste identique ! 
</center>
3 paramètres : $\alpha,\beta,\gamma  $

$\alpha$ est l'inverse de la période d'incubation (1/incubation)

$\beta$ est le taux moyen de contact de la population

$\gamma$ est l'inverse de la période moyenne d'infection (1/infection)

Autre valeur importante : le $R_0$ qui montre à quelle vitesse la maladie se répend et peut être calculé de la manière suivante : 


<center>$R_0 = \frac{\beta}{\gamma}$ (6)</center>
    
A nous par la suite de définir une période d'incubation. Ajd, ca semble être 5 jours donc $\alpha$ semble être aux alentours de 0.2. 

Le $R_0$ est semblerait il d'après plusieurs papiers autour de 3.5 . A voir en France.

Et à l'aide d'un $\gamma$ défini nous pouvons avec (6) retrouvé $\beta$


### Ajout d'un effet de distance sociale

Cette fois ci , ajout d'un effet de social distancing qui va avoir un impact sur notre $\beta$ variable mesurant le taux de contact entre les personnes.

Soit $\rho$ la distanciation sociale variant entre 0 et 1. Où 0 est tout le monde est confiné, 1 tout le monde circule comme il le souhaite. Pour introduire ce dernier facteur, on va simplement changer nos deux premières équations en : 

<center> $\dot{S} = -\rho \beta SI$ (1') </center>

<center> $\dot{E} = \rho \beta SI - \alpha E $ (2') </center>

In [22]:
def SEIR_mano(val_init,coeff,t):
    """
        Input : 
            val_init : Liste de 4 valeurs initials que va prendre le modèle 
            coeff : Liste de valeurs pour Alpha,Gamma, Beta et Rho
            t :
        Output:
            
    """
    S0,E0,I0,R0= val_init
    S,E,I,R = [S0],[E0],[I0],[R0]
    a,b,g,r=coeff
    dt = t[1]-t[0]
    for _ in t[1:]:
        nS = S[-1] - (r*b*S[-1]*I[-1])*dt
        nE = E[-1] + (r*b*S[-1]*I[-1]-a*E[-1])*dt
        nI = I[-1] + (a*E[-1]-g*I[-1])*dt
        nR = R[-1] + (g*I[-1])*dt
        S.append(nS)
        E.append(nE)
        I.append(nI)
        R.append(nR)
    return np.stack([S,E,I,R]).T

In [25]:
t = np.linspace(0, 100, int(100/0.1) + 1)
incub_time = 5
infec_time = 11
alpha = 1/incub_time
R0 = 3.2
gamma = 1/infec_time
beta = R0 * gamma
rho = 1
N = 6000
exposed = 1
coeff = alpha, beta,gamma,rho
print(coeff)
init_vals = 1 - exposed/N,1/N,0,0

(0.2, 0.29090909090909095, 0.09090909090909091, 1)


In [28]:
df = pd.DataFrame(SEIR_mano(init_vals,coeff,t),columns=['S','E','I','R'])